# Feature Engineering

### Packages and Dependencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer

from textblob import TextBlob
from textblob import Word

stop = stopwords.words('english')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package stopwords to /home/tom/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load Data

In [3]:
main = pd.read_csv('main.csv').drop('Unnamed: 0', 1)

/home/tom/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (343,344,351,352,354,356,357,358,359,360,361,362,371,372) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Inspect

In [4]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7033 entries, 0 to 7032
Columns: 372 entries, ep_id to league_10
dtypes: float64(300), int64(1), object(71)
memory usage: 20.0+ MB


In [6]:
main.sample(10)

,ep_id,full_name,date_of_birth,hometown,country,youth_team,position,height,weight,shoots,status,scout_text,regular_gp_17,regular_gp_18,regular_gp_19,regular_gp_20,regular_gp_21,regular_gp_22,regular_gp_23,regular_gp_24,regular_gp_25,regular_gp_26,regular_gp_27,regular_gp_28,regular_g_17,regular_g_18,regular_g_19,regular_g_20,regular_g_21,regular_g_22,regular_g_23,regular_g_24,regular_g_25,regular_g_26,regular_g_27,regular_g_28,regular_a_17,regular_a_18,regular_a_19,regular_a_20,regular_a_21,regular_a_22,regular_a_23,regular_a_24,regular_a_25,regular_a_26,regular_a_27,regular_a_28,regular_pim_17,regular_pim_18,regular_pim_19,regular_pim_20,regular_pim_21,regular_pim_22,regular_pim_23,regular_pim_24,regular_pim_25,regular_pim_26,regular_pim_27,regular_pim_28,regular_pm_17,regular_pm_18,regular_pm_19,regular_pm_20,regular_pm_21,regular_pm_22,regular_pm_23,regular_pm_24,regular_pm_25,regular_pm_26,regular_pm_27,regular_pm_28,playoffs_g_17,playoffs_g_18,playoffs_g_19,playoffs_g_20,playoffs_g_21,playoffs_g_22,playoffs_g_23,playoffs_g_24,playoffs_g_25,playoffs_g_26,playoffs_g_27,playoffs_g_28,playoffs_a_17,playoffs_a_18,playoffs_a_19,playoffs_a_20,playoffs_a_21,playoffs_a_22,playoffs_a_23,playoffs_a_24,playoffs_a_25,playoffs_a_26,playoffs_a_27,playoffs_a_28,playoffs_pim_17,playoffs_pim_18,playoffs_pim_19,playoffs_pim_20,playoffs_pim_21,playoffs_pim_22,playoffs_pim_23,playoffs_pim_24,playoffs_pim_25,playoffs_pim_26,playoffs_pim_27,playoffs_pim_28,playoffs_pm_17,playoffs_pm_18,playoffs_pm_19,playoffs_pm_20,playoffs_pm_21,playoffs_pm_22,playoffs_pm_23,playoffs_pm_24,playoffs_pm_25,playoffs_pm_26,playoffs_pm_27,playoffs_pm_28,award_count_17,award_count_18,award_count_19,award_count_20,award_count_21,award_count_22,award_count_23,award_count_24,award_count_25,award_count_26,award_count_27,award_count_28,regular_gp_16,regular_gp_29,regular_gp_30,regular_gp_31,regular_gp_32,regular_gp_33,regular_gp_34,regular_gp_35,regular_g_16,regular_g_29,regular_g_30,regular_g_31,regular_g_32,regular_g_33,regular_g_34,regular_g_35,regular_a_16,regular_a_29,regular_a_30,regular_a_31,regular_a_32,regular_a_33,regular_a_34,regular_a_35,regular_pim_16,regular_pim_29,regular_pim_30,regular_pim_31,regular_pim_32,regular_pim_33,regular_pim_34,regular_pim_35,regular_pm_16,regular_pm_29,regular_pm_30,regular_pm_31,regular_pm_32,regular_pm_33,regular_pm_34,regular_pm_35,playoffs_g_16,playoffs_g_29,playoffs_g_30,playoffs_g_31,playoffs_g_32,playoffs_g_33,playoffs_g_34,playoffs_g_35,playoffs_a_16,playoffs_a_29,playoffs_a_30,playoffs_a_31,playoffs_a_32,playoffs_a_33,playoffs_a_34,playoffs_a_35,playoffs_pim_16,playoffs_pim_29,playoffs_pim_30,playoffs_pim_31,playoffs_pim_32,playoffs_pim_33,playoffs_pim_34,playoffs_pim_35,playoffs_pm_16,playoffs_pm_29,playoffs_pm_30,playoffs_pm_31,playoffs_pm_32,playoffs_pm_33,playoffs_pm_34,playoffs_pm_35,award_count_16,award_count_29,award_count_30,award_count_31,award_count_32,award_count_33,award_count_34,award_count_35,regular_gp_15,regular_gp_36,regular_g_15,regular_g_36,regular_a_15,regular_a_36,regular_pim_15,regular_pim_36,regular_pm_15,regular_pm_36,playoffs_g_15,playoffs_g_36,playoffs_a_15,playoffs_a_36,playoffs_pim_15,playoffs_pim_36,playoffs_pm_15,playoffs_pm_36,award_count_15,award_count_36,regular_gp_37,regular_gp_38,regular_gp_39,regular_g_37,regular_g_38,regular_g_39,regular_a_37,regular_a_38,regular_a_39,regular_pim_37,regular_pim_38,regular_pim_39,regular_pm_37,regular_pm_38,regular_pm_39,playoffs_g_37,playoffs_g_38,playoffs_g_39,playoffs_a_37,playoffs_a_38,playoffs_a_39,playoffs_pim_37,playoffs_pim_38,playoffs_pim_39,playoffs_pm_37,playoffs_pm_38,playoffs_pm_39,award_count_37,award_count_38,award_count_39,regular_gp_14,regular_g_14,regular_a_14,regular_pim_14,regular_pm_14,playoffs_g_14,playoffs_a_14,playoffs_pim_14,playoffs_pm_14,award_count_14,regular_gp_13,regular_g_13,regular_a_13,regular_pim_13,regular_pm_13,playoffs_g_13,playoffs_a_13,playoffs_pim_13,playoffs_pm_13,award_count_13,regular_gp_12,regular_g_12,regul

### Refactor league_25 as the response

In [ ]:
replacements = {
    np.nan: 'Not Active',
    0: 'Not Active',
    'ECHL': 'Lower',
    'AHL': 'Lower',
    'USports': 'Lower', # Collegiate
    'NCHL': 'Lower', # Collegiate
    'LHSAM': 'Lower',
    'LHCS': 'Lower',
    'LNAH': 'Lower',
    'CHL': 'Lower',
    'NCHL-MB': 'Lower',
    'NLSHL': 'Lower',
    'SPHL': 'Lower',
    'LHSR': 'Lower',
    'LHSCE': 'Lower',
    'SEMHL': 'Lower',
    'NESHL': 'Lower',
    'WOAA': 'Lower',
    'HHL': 'Lower',
    'CRL': 'Lower',
    'FPHL': 'Lower',
    'Chinook HL': 'Lower',
    'FHL': 'Lower',
    'LHSAAAQ': 'Lower',
    'BSHL': 'Lower',
    'NPHL-Sr.': 'Lower',
    'WMHL': 'Lower',
    'ACHW': 'Lower',
    'GLHL': 'Lower',
    'RHL': 'Lower',
    'ACAC': 'Lower',
    'SVHL': 'Lower',
    'SWHL': 'Lower',
    'BCIHL': 'Lower',
    'QVHL': 'Lower',
    'WCSHL': 'Lower',
    'LHBBF': 'Lower',
    'OHL': 'Lower',
    'BRHL': 'Lower',
    'FCHL': 'Lower',
    'THHL': 'Lower',
    'ACHA II': 'Lower',
    'LHSLF': 'Lower',
    'CIHL': 'Lower',
    'LRH': 'Lower',
    'NSHL-Sr.': 'Lower',
    'LHSEQ': 'Lower',
    'ACH': 'Lower',
    'IHL': 'Lower',
    'MSHL': 'Lower',
    'SASHL': 'Lower',
    'CSHL-Sr.': 'Lower',
    'USHL': 'Lower',
    'OWSHL': 'Lower',
    'LLHL': 'Lower',
    'CWSHL': 'Lower',
    'LHSBEQ': 'Lower',
    'QSCHL': 'Lower',
    'LHSAO': 'Lower',
    'ASHL': 'Lower',
    'NCAA III': 'Lower',
    'THL': 'Lower',
    'AAHL': 'Lower',
    'Notekeu HL': 'Lower',
    'QMJHL': 'Lower',
    'ACHA III': 'Lower',
    'LHSCN': 'Lower',
    'EPHL': 'Lower'
}

main['response'] = main['league_25'].replace(replacements)

### Refactor country as birth_region

In [ ]:
regions = {
    'Czech': 'Eastern Europe',
    'Sweden': 'Scandanavia',
    'Germany': 'Western Europe',
    'Slovakia': 'Eastern Europe',
    'Finland': 'Scandanavia',
    'Italy': 'Western Europe',
    'Norway': 'Scandanavia',
    'Switzerland': 'Western Europe',
    'Latvia': 'Eastern Europe',
    'Ukraine': 'Eastern Europe',
    'Denmark': 'Scandanavia',
    'Hungary': 'Eastern Europe',
    'Belarus': 'Eastern Europe',
    'Kazakhstan': 'Eastern Europe',
    'Slovenia': 'Eastern Europe',
    'U': 'Western Europe', # U is United Kingdom (UK)
    'Austria': 'Western Europe',
    'France': 'Western Europe',
    'South': 'Asia',
    'Croatia': 'Eastern Europe',
    'Israel': 'Eastern Europe',
    'Poland': 'Eastern Europe',
    'England': 'Western Europe',
    'Japan': 'Asia',
    'Netherlands': 'Western Europe'
}

main['birth_region'] = main['country'].replace(regions)

### Manage position

In [ ]:
positions = {
    'RW/C': 'Forward',
    'C/LW': 'Forward',
    'RW/LW': 'Winger',
    'D/F': 'Mixed',
    'F': 'Forward',
    'LW/D': 'Mixed',
    'C/RW': 'Forward',
    'LW/RW': 'Forward',
    'LW/C': 'Forward',
    'LW/C/D': 'Mixed',
    'D/W': 'Mixed',
    'W/C/D': 'Mixed',
    'W/C': 'Forward',
    'C/W': 'Forward',
    'D/RW': 'Mixed',
    'F/D': 'Mixed',
    'D/LW': 'Mixed',
    'RW/D': 'Mixed',
    'D/C': 'Mixed',
    'C/RW/D': 'Mixed',
    'W/D': 'Mixed',
    'W': 'Forward',
    'C/D': 'Mixed'
}

main['position'] = main['position'].replace(positions)

### Subset and reorder

In [ ]:
reorder = ['ep_id',
    'height', 'weight', 'games_played_14', 'goals_14',
    'assists_14', 'penalty_min_14', 'plus_minus_14', 
    'games_played_15', 'goals_15', 'assists_15', 
    'penalty_min_15', 'plus_minus_15', 
    'games_played_16', 'goals_16', 'assists_16', 'penalty_min_16', 
    'plus_minus_16', 'games_played_17', 'goals_17', 
    'assists_17', 'penalty_min_17', 'plus_minus_17', 
    'scout_text', 'birth_region', 'position', 'shoots', 'response'
]

main = main[reorder]

In [ ]:
main = main.loc[main['weight'] > 40]

In [ ]:
sns.countplot(y=main['response'])

In [ ]:
round(main.response.value_counts() / len(main), 2)

In [ ]:
fig, ax = plt.subplots(1, figsize=[12.5, 15])

ax = sns.heatmap(main.isnull(), cbar=False)

plt.show()

### Impute values

In [ ]:
main.columns

In [ ]:
counts = [
    'games_played_14', 'goals_14', 'assists_14', 'penalty_min_14',
    'games_played_15', 'goals_15', 'assists_15', 'penalty_min_15',
    'games_played_16', 'goals_16', 'assists_16', 'penalty_min_16',
    'games_played_17', 'goals_17', 'assists_17', 'penalty_min_17',
]

plus_minuses = [
    'plus_minus_14', 'plus_minus_15', 'plus_minus_16', 'plus_minus_17'
]

In [ ]:
main[counts] = main[counts].replace(np.nan, 0)
main[plus_minuses] = main[plus_minuses].replace(np.nan, 0)

In [ ]:
fig, ax = plt.subplots(1, figsize=[12.5, 7])

ax = sns.heatmap(main.isnull(), cbar=False)

plt.show()

## Scouting Reports

In [ ]:
partial = pd.read_csv('scout_text_partial_label.csv').drop('Unnamed: 0', 1)
partial = partial[partial['type'] == 'Yes']
response = main[['ep_id', 'response']]
partial = pd.merge(partial, response, on='ep_id', how='left')

In [ ]:
partial.info()

In [ ]:
def avg_words(sentence):
    words = sentence.split()
    return round((sum(len(word) for word in words)/len(words)), 2)

In [ ]:
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
partial['scout_text'] = partial['scout_text'].str.replace('[^\w\s]','')
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))

In [ ]:
partial['word_count'] = partial['scout_text'].apply(lambda x: len(str(x).split(' ')))
partial['char_count'] = partial['scout_text'].str.len()
partial['avg_word'] = partial['scout_text'].apply(lambda x: avg_words(x))
partial['stop_words'] = partial['scout_text'].apply(lambda x: len([x for x in x.split() if x in stop]))
partial['upper'] = partial['scout_text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [ ]:
freq = pd.Series(' '.join(partial['scout_text']).split()).value_counts()[:10]

In [ ]:
freq = list(freq.index)
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join(x for x in x.split() if x not in freq))

In [ ]:
freq = pd.Series(' '.join(partial['scout_text']).split()).value_counts()[-1636:]

In [ ]:
freq = list(freq.index)
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join(x for x in x.split() if x not in freq))

In [ ]:
#partial['scout_text'] = partial['scout_text'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
TextBlob(partial['scout_text'][1]).words

In [ ]:
st = PorterStemmer()
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join([st.stem(word) for word in x.split()]))

In [ ]:
partial['scout_text'] = partial['scout_text'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))

In [ ]:
partial['sentiment'] = partial['scout_text'].apply(lambda x: TextBlob(x).sentiment[0])

In [ ]:
partial.info()

In [ ]:
main.info()

In [ ]:
sns.distplot(partial['sentiment'])

In [ ]:
sns.boxplot(partial['response'], partial['sentiment'])

In [ ]:
keep = [
    'ep_id', 'type', 'word_count', 'char_count', 'avg_word', 'stop_words', 'upper', 'sentiment'
]

partial = partial[keep]

In [ ]:
main = pd.merge(main, partial, on='ep_id', how='left')

In [ ]:
main = main[main['type'] == 'Yes']

In [ ]:
sns.heatmap(main.isnull(), cbar=False)

In [ ]:
main.info()

### Inspect and Export

In [ ]:
main_test = main.drop(['ep_id', 'scout_text', 'type'], 1)

In [ ]:
main_test.info()

In [ ]:
#main_test.to_csv('main_test.csv')